# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

In [3]:
MODEL = "llama3.2"

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
def get_links_with_ollama(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ]
    )
    
    result = response['message']['content']
    print(type(result))
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

edwarddonner = Website("https://edwarddonner.com")
edwarddonner.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [13]:
get_links_with_ollama("https://edwarddonner.com")

<class 'str'>


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links_with_ollama(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://edwarddonner.com"))

<class 'str'>
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'home page', 'url': 'https://edwarddonner.com/'}, {'type': 'Careers/Jobs page', 'url': 'https://edwarddonner.com/'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'}]}
Landing page:
Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and purs

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("EdwarDdonner", "https://edwarddonner.com")

<class 'str'>
Found links: {'links': [{'type': 'home page', 'url': 'https://edwarddonner.com'}, {'type': 'about me page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'careers/jobs page', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'blog posts page', 'url': 'https://edwarddonner.com/posts/'}]}


'You are looking at a company called: EdwarDdonner\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m pr

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
def create_brochure_with_ollama(company_name, url):
    response = ollama.chat(
        model=MODEL,
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
      ],
    )
    
    result = response['message']['content']
    return result
    # display(Markdown(result))

In [21]:
result = create_brochure_with_ollama("EdwarDdonner", "https://edwarddonner.com")
display(Markdown(result))

<class 'str'>
Found links: {'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'}, {'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'blog posts', 'url': 'https://edwarddonner.com/posts/'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'}, {'type': 'Facebook profile', 'url': 'https://www.facebook.com/edward.donner.52'}]}


**Welcome to Edward Donner**

[Image: A futuristic illustration of artificial intelligence and humans working together]

**Our Mission**

At Edward Donner, we're passionate about applying AI to make a positive impact on people's lives. Our mission is to help individuals discover their potential and pursue their reason for being, leading to greater fulfillment and success.

**About Us**

We're the co-founders of Nebula.io, an AI-powered platform that helps recruiters source, understand, engage, and manage talent. Our patented matching model uses Generative AI to match people with roles without relying on keywords, making it faster and more accurate than ever before.

**Our Story**

Our journey began in 2013 when our founder founded an AI startup called untapt, which specialized in building talent marketplaces and data science software for recruitment firms. We later acquired this company in 2021 and expanded our focus to apply AI to real-world problems, particularly in the hiring process.

**What We Do**

* Develop groundbreaking LLMs verticalized for talent
* Patent our matching model for more accurate and efficient matching
* Create an award-winning platform that has happy customers and tons of press coverage

**Connect with Us**

Stay up-to-date with our latest news, courses, and updates by following us on:

* LinkedIn: [link]
* Twitter: [link]
* Facebook: [link]

Join the conversation and subscribe to our newsletter to stay informed about our mission and impact.

**Get Involved**

Becoming an LLM expert and leader is easier than ever. Check out our courses, such as "Connecting my courses" and "The Complete Agentic AI Engineering Course".

[Call-to-Action: Subscribe to Newsletter | Follow Us on Social Media]

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
def stream_brochure_with_ollama(company_name, url):
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.get("message", {}).get("content", "")
        cleaned = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure_with_ollama("EdwarDdonner", "https://edwarddonner.com")

<class 'str'>
Found links: {'links': [{'type': 'home page', 'url': 'https://edwarddonner.com'}, {'type': 'about me and about Nebula', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'careers/jobs page', 'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}, {'type': 'home page', 'url': 'https://edwarddonner.com'}, {'type': 'press release link', 'url': 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html'}]}


# Edward Donner Brochure

## Welcome to Edward Donner

At Edward Donner, we're revolutionizing the way people discover their potential and pursue their reason for being. Our mission is to help individuals find roles that align with their passions and values, leading to a more fulfilled and successful life.

### Who We Are

We're a team of passionate innovators driven by a shared vision to harness the power of artificial intelligence (AI) to solve real-world problems. Our co-founder and CTO, Ed, has over two decades of experience in AI and talent acquisition, having founded and sold his previous startup, untapt, in 2021.

### Our Mission

We're committed to helping people find their Ikigai – that sweet spot where personal passion meets purpose. By applying AI to the hiring process, we've developed a patented matching model that connects talented individuals with roles that align with their skills and aspirations. Our long-term goal is to raise human prosperity by empowering people to pursue their dreams.

### How We Do It

We use cutting-edge Generative AI and machine learning technologies to create personalized talent pipelines for recruiters. Our platform streamlines the hiring process, reducing the time and effort required to find top talent. With Edward Donner, you can:

* Source high-quality candidates without relying on keywords
* Understand candidate motivations and preferences
* Engage with top performers in real-time
* Manage your talent pipeline efficiently

### Join Our Community

Stay up-to-date with our latest news, insights, and updates by following us on LinkedIn, Twitter, or Facebook. Subscribe to our newsletter to receive exclusive content, tips, and resources.

### Get Involved

Ready to take the first step towards finding your Ikigai? Explore our free trial of our platform today and discover how Edward Donner can help you achieve your career goals.

Contact Us
ed [at] edwarddonner [dot] com
www.edwarddonner.com

In [25]:
translator_system_prompt = "You are a professional translator. Your task is to translate the provided company brochure from English to Vietnamese. \
Preserve the original structure and formatting, including any markdown sections, headings, or bullet points. \
Use natural, fluent, and professional Vietnamese suitable for prospective customers, investors, and job seekers in Vietnam. \
Avoid literal word-by-word translation — prioritize clarity, accuracy, and natural flow in Vietnamese."

In [26]:
def get_translation_user_prompt(brochure_content):
    user_prompt = "Please translate the following company brochure into Vietnamese.\n"
    user_prompt += "Preserve all formatting and structure (e.g., markdown headings, lists, etc).\n"
    user_prompt += "Use natural, fluent, and professional Vietnamese suitable for prospective customers, investors, and job seekers.\n"
    user_prompt += "\nBrochure content to translate:\n"
    user_prompt += brochure_content
    user_prompt = user_prompt[:5_000]  # Truncate if needed
    return user_prompt

In [27]:
def stream_translated_brochure_with_ollama(company_name, url):
    brochure_content = create_brochure_with_ollama(company_name, url)
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": translator_system_prompt},
            {"role": "user", "content": get_translation_user_prompt(brochure_content)},
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.get("message", {}).get("content", "")
        cleaned = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)

In [28]:
stream_translated_brochure_with_ollama("EdwarDdonner", "https://edwarddonner.com")

<class 'str'>
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}, {'type': 'careers page', 'url': 'https://edwarddonner.com/posts/'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'}]}


**Khám phá Edward Donner**
=====================

**Xin chào Edward Donner**

Chúng tôi là một công ty đam mê cách thức Harnessing Artificial Intelligence (AI) để tạo ra tác động tích cực cho cuộc sống của con người. Là co-founder và CTO của Nebula.io, sứ mệnh của chúng tôi là giúp các cá nhân khám phá tiềm năng của mình và theo đuổi lý do tại sao họ lại tồn tại.

**Câu chuyện của chúng tôi**

Tại Edward Donner, chúng tôi được thúc đẩy bởi một khái niệm gọi là Ikigai, có nghĩa là tìm kiếm mục đích và thỏa mãn trong cuộc sống. Mục tiêu dài hạn của chúng tôi là giúp con người tìm ra vai trò phù hợp với đam mê và giá trị của mình, dẫn đến sự thịnh vượng con người tăng lên.

Sự khởi đầu của hành trình này bắt nguồn từ một công ty khởi nghiệp AI tên untapt được thành lập vào năm 2013. Chúng tôi phát triển các khu vực trao đổi tài năng và phần mềm khoa học dữ liệu cho các công ty tuyển dụng, chuyên về vai trò công nghệ trong lĩnh vực tài chính. Thành công của chúng tôi đã thu hút sự chú ý của các tổ chức uy tín, bao gồm Accenture FinTech Innovation Lab và American Banker, người đã đặt chúng vào danh sách những công ty cần theo dõi.

**Công nghệ của chúng tôi**

Chúng tôi đã cấp bằng sáng chế cho mô hình khớp nối của mình, sử dụng Generative AI và học máy để khớp nối người với vai trò có độ chính xác và tốc độ cao hơn bao giờ hết. Mạng lưới này được thiết kế để giúp các nhà tuyển dụng tìm kiếm, hiểu biết, tương tác và quản lý nhân sự một cách hiệu quả hơn.

**Connect Four: Arena**

Nhưng chúng tôi không chỉ về công nghệ – chúng tôi cũng có một mặt giải trí! Connect Four là một sân chơi nơi Các Model Ngôn ngữ To lớn (LLMs) cạnh tranh trong một cuộc chiến sự minh bạch và tinh vi. Đó là một không gian độc đáo để con người tương tác với AI và khám phá tiềm năng của ngôn ngữ và thông minh.

**Lựa chọn tham gia cộng đồng**

Nếu bạn quan tâm đến việc học thêm về Edward Donner hoặc muốn kết nối với chúng tôi, xin vui lòng liên hệ qua trang web hoặc kênh truyền thông xã hội của chúng tôi. Chúng tôi rất hân hạnh được nghe từ bạn!

**Làm việc cho đội ngũ**

Chúng tôi luôn tìm kiếm những cá nhân tài năng để gia nhập nhóm của chúng tôi. Vui lòng kiểm tra trang sự nghiệp của chúng tôi để biết thêm về các vị trí có sẵn và cách bạn có thể đóng góp vào sứ mệnh của chúng tôi.

**Trên đây, cập nhật**

Đăng ký tin nhắn của chúng tôi để được cập nhật thông tin mới nhất, thông tin cập nhật và những hiểu biết sâu sắc từ Edward Donner.

[Tương tác trên mạng xã hội]

[Tư vấn liên lạc]

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>